#Linear Programming for Stock Optimization


In many business applications, prediction is only the starting point. For instance, if we have a model that predicts stocks in different warehouses, we still need to see how to best optimize the transport between the warehouses. This notebook shows how to optimize tranport in a network using linear programming with Pyomo.  

We will consider a network of warehouses. Each warehouses has a given stock capacity and a transport cost attached to each available route between warehouses. We wish to satisfy the demands in each target warehouses.

***

## Setup and dataset loading  
First of all, let's load the libraries that we'll use.

**This notebook requires the installation of the [pyOmo](https://pyomo.readthedocs.io/en/latest/) package.** [See here for help with intalling python packages.](https://www.dataiku.com/learn/guide/code/python/install-python-packages.html)

**You should also check that an optimization solver (glpk for example) is alreayd installed.**

In [1]:
%pylab inline
import warnings 
warnings.filterwarnings("ignore",category=DeprecationWarning) # Disable some warnings
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd

from pyomo.environ import *

Populating the interactive namespace from numpy and matplotlib


In [2]:
dataset_limit = 10000

#PUT YOUR DATASET NAME HERE 
mydataset = dataiku.Dataset("Customer_Analysis_prepared")
df = mydataset.get_dataframe(limit = dataset_limit)

Here we need to precise the structure of the network. We have edges of two types: source warehouses (*source_warehouse*) and target warehouses (*target_warehouse*), and vertices are weighted by the transport cost (*transport_cost*) between two warehouses. We also need the capacity at each warehouse (*capacity_warehouse*) as well as its demand (*demand_warehouse*).

In [3]:
#Below enter the name of the corresponding features
source_warehouse = ''
target_warehouse = ''
transport_cost = ''
capacity_warehouse = ''
demand_warehouse = ''

***

## Model Formulation

A Linear Programming problem aims at finding the minimum of a **linear objective function** subject to a set of **linear constraints** on some **variables**.  

Here we wish to minimise the overall cost of transport with **three constraints**:
    
    1) The transported quantities must be non-negative. 
    2) A source warehouse cannot ship more than what it contains. 
    3) We need to satisfy demands in our target warehouses.   

Let's make this more concrete by using pyomo's module API. We'll start by instantiating a **ConcreteModel**.

In [4]:
model = ConcreteModel()

Pyomo supports an object-oriented design for the definition of optimization models. A Pyomo model object contains a collection of modeling components that define the optimization problem.These modeling components are defined in Pyomo through the following Python classes:

* **Var**: optimization variables in a model.
* **Objective**: functions that are minimized or maximized in a model.
* **Constraint**: constraint equations in a model.
* **Set**: set data that is used to define a model instance.
* **Param**: parameter data that is used to formulate a constraint or objective.

We will start by instantiating the variables and then use them in the constraint equations and objective function.

### Warehouse Index Sets  
We define two index sets with *Set()*: **S** for the set of source warehouses and **T** for the target warehouses. 

In [5]:
model.s = Set(initialize=list(df[source_warehouse].unique()))
model.t = Set(initialize=list(df[target_warehouse].unique()))

KeyError: ''

### Quantity Variables  
We denote by $x_{S,T}$ the desired quantity variable of goods that needs to be shipped from source warehouse **S** to target warehouse **T**. Note that we only consider non-negative quantities and so we add a lower bound to the variable.

In [ ]:
model.x = Var(model.s, model.t, bounds=(0.0,None))

### Capacity and Demand Variables  
Each source warehouses has a capacity $a_S$ and each target warehouse a demand $b_T$ which are encoded in the two dictionnaries below. We can then define the two corresponding model parameters for our model.

In [ ]:
dict_a = dict(zip(df[source_warehouse], df[capacity_warehouse]))
dict_b = dict(zip(df[target_warehouse], df[demand_warehouse]))

model.a = Param(model.s, initialize=dict_a)
model.b = Param(model.t, initialize=dict_b)

### Cost Variables  
For a pair of *(source warehouse, target warehouse)*, we have an associated cost of transport per unit of goods. 

We first get its corresponding dictionary from our dataframe with:

In [ ]:
source_target_cost = {}
for index, row in df.iterrows():
    source_target_cost[row[source_warehouse], row[target_warehouse]] = row[transport_cost]

We can know define its corresponding parameter in our model.

In [ ]:
model.c = Param(model.s, model.t, initialize=source_target_cost)

### Objective Function  
Every optimisation problem  requires an objective function. Here we'll consider the linear function of total cost (so that we effectively have to solve a Linear Optimisation problem) defined as follows. Our objective is to minimise this cost.

In [ ]:
def objective_rule(model):
    return sum(model.c[s,t]*model.x[s,t] for s in model.s for t in model.t if (s,t) in model.c.keys())

In [ ]:
model.objective = Objective(rule=objective_rule, sense=minimize)

### Constraints  
The above objective is subject to the following constraints: 
* The sum of all the quantities leaving a source warehouse cannot exceed its capacity.
* The sum of all the quantities arriving at a target warehouse should be at least its demand. 

In [ ]:
def supply_rule(model, s):
    return sum(model.x[s,t] for t in model.t) <= model.a[s]

def demand_rule(model, t):
    return sum(model.x[s,t] for s in model.s) >= model.b[t]  


In [ ]:
model.supply = Constraint(model.s, rule=supply_rule)
model.demand = Constraint(model.t, rule=demand_rule)

***

## Running the Optimization  
Now that our model has been well specified and declared as a linear optimisation problem, we can try to solve it. Here we used the [glpk solver](https://www.gnu.org/software/glpk/).

In [ ]:
from pyomo.opt import SolverFactory
opt = SolverFactory("glpk")
results = opt.solve(model)

## Results  
We can know check the run of our optimization:

In [ ]:
results.write_json()

Finally, we can check the optimal (if it exists) vector of quantities between warehouses.

In [ ]:
dict_optim = model.x.extract_values()

Better yet, we will add the found optimal transport values to our initial dataframe.

In [ ]:
source,target = zip(*dict_optim)
optim_values = [dict_optim[source[i], target[i]] for i in xrange(len(dict_optim))]

model_output = pd.DataFrame({source_warehouse: source, target_warehouse: target, 'optim_values' : optim_values})
model_output.head()

We then have to merge this with the original dataframe.

In [ ]:
doptim_= pd.merge(df, model_output, on=[source_warehouse,target_warehouse], how='inner')
doptim_.head()